In [1]:
library("DEP")
library("dplyr")

Warning message:
"package 'DEP' was built under R version 3.5.2"
Warning message in fun(libname, pkgname):
"mzR has been built against a different Rcpp version (1.0.0)
than is installed on your system (1.0.3). This might lead to errors
when loading mzR. If you encounter such issues, please send a report,
including the output of sessionInfo() to the Bioc support forum at 
https://support.bioconductor.org/. For details see also
https://github.com/sneumann/mzR/wiki/mzR-Rcpp-compiler-linker-issue."
Warning message:
"package 'dplyr' was built under R version 3.5.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
# type = "LFQ.intensity.NOT_" ; target_path = "LFQ_DEP_DE.csv"
# type = "iBAQ.NOT_"; target_path = "IBAQ_DEP_DE.csv"
type = "Intensity.NOT_"; target_path = "INTENSITY_DEP_DE.csv"

file <- '../../data//20150718_Cerberus_proteinGroups.txt'
ed_file <-'../../data//profile_experimental_design.csv'

data <- read.csv(file, sep = "\t", stringsAsFactors = FALSE)
experimental_design <- read.table(ed_file, header=TRUE, sep="\t", stringsAsFactors = FALSE)
experimental_design <- read.table(ed_file, header=TRUE, sep=";", stringsAsFactors = FALSE)
experimental_design$X <- NULL

In [3]:
experimental_design$label <- sapply(experimental_design$label,function(x) paste0(type,strsplit(x,"NOT_",fixed = T)[[1]][2]))

In [4]:
data <- filter(data, Reverse != "+", Potential.contaminant != "+")
data_unique <- make_unique(data,"Majority.protein.IDs" ,"Gene.names",delim = " ")

In [5]:
LFQ_columns <- grep(type, colnames(data_unique))
data_se <- make_se(data_unique, LFQ_columns, experimental_design)

data_filt <- filter_missval(data_se, thr = 0)

In [6]:
data_norm <- normalize_vsn(data_filt)
data_imp <- impute(data_norm, fun = "MinProb", q = 0.01)

Loading required package: imputeLCMD

Warning message:
"package 'imputeLCMD' was built under R version 3.5.3"
Loading required package: tmvtnorm

Warning message:
"package 'tmvtnorm' was built under R version 3.5.3"
Loading required package: mvtnorm

Warning message:
"package 'mvtnorm' was built under R version 3.5.3"
Loading required package: Matrix

Warning message:
"package 'Matrix' was built under R version 3.5.3"
Loading required package: stats4

Loading required package: gmm

Warning message:
"package 'gmm' was built under R version 3.5.3"
Loading required package: sandwich

Warning message:
"package 'sandwich' was built under R version 3.5.3"
Loading required package: norm

Warning message:
"package 'norm' was built under R version 3.5.2"
Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':


[1] 1.389606


In [7]:
data_diff_all_contrasts <- test_diff(data_imp,  type = "all")
dep <- add_rejections(data_diff_all_contrasts, alpha = 0.05, lfc = log2(1.5))

data_results <- get_results(dep)

Tested contrasts: BCellmemory_vs_BCellnaive, BCellmemory_vs_BCellplasma, BCellmemory_vs_CD4TCellmTregs, BCellmemory_vs_CD4TCellnaive, BCellmemory_vs_CD4TCellnTregs, BCellmemory_vs_CD4TCellTcm, BCellmemory_vs_CD4TCellTem, BCellmemory_vs_CD4TCellTemra, BCellmemory_vs_CD4TCellTh1, BCellmemory_vs_CD4TCellTh17, BCellmemory_vs_CD4TCellTh2, BCellmemory_vs_CD8TCellnaive, BCellmemory_vs_CD8TCellTcm, BCellmemory_vs_CD8TCellTem, BCellmemory_vs_CD8TCellTemra, BCellmemory_vs_DendriticCD1c, BCellmemory_vs_DendriticCD304, BCellmemory_vs_GranulocytesBasophil, BCellmemory_vs_Granulocyteseosinophils, BCellmemory_vs_Granulocytesneutrophils, BCellmemory_vs_Monocytesclassical, BCellmemory_vs_Monocytesintermediate, BCellmemory_vs_Monocytesnonclassical, BCellmemory_vs_NKCellsCD56bright, BCellmemory_vs_NKCellsCD56dim, BCellmemory_vs_Erythrocytes, BCellmemory_vs_Thrombocytes, BCellnaive_vs_BCellplasma, BCellnaive_vs_CD4TCellmTregs, BCellnaive_vs_CD4TCellnaive, BCellnaive_vs_CD4TCellnTregs, BCellnaive_vs_CD4TCe

In [8]:
# write.csv2(data_results,target_path)

In [10]:
colnames(data_results)

[1] "name"                                                          
   [2] "ID"                                                            
   [3] "BCellmemory_vs_BCellnaive_p.val"                               
   [4] "BCellmemory_vs_BCellplasma_p.val"                              
   [5] "BCellmemory_vs_CD4TCellmTregs_p.val"                           
   [6] "BCellmemory_vs_CD4TCellnaive_p.val"                            
   [7] "BCellmemory_vs_CD4TCellnTregs_p.val"                           
   [8] "BCellmemory_vs_CD4TCellTcm_p.val"                              
   [9] "BCellmemory_vs_CD4TCellTem_p.val"                              
  [10] "BCellmemory_vs_CD4TCellTemra_p.val"                            
  [11] "BCellmemory_vs_CD4TCellTh1_p.val"                              
  [12] "BCellmemory_vs_CD4TCellTh17_p.val"                             
  [13] "BCellmemory_vs_CD4TCellTh2_p.val"                              
  [14] "BCellmemory_vs_CD8TCellnaive_p.val"                            
  [15] "BCellmemory_vs_CD8TCellTcm_p.val"                              
  [16] "BCellmemory_vs_CD8TCellTem_p.val"                              
  [17] "BCellmemory_vs_CD8TCellTemra_p.val"                            
  [18] "BCellmemory_vs_DendriticCD1c_p.val"                            
  [19] "BCellmemory_vs_DendriticCD304_p.val"                           
  [20] "BCellmemory_vs_Erythrocytes_p.val"                             
  [21] "BCellmemory_vs_GranulocytesBasophil_p.val"                     
  [22] "BCellmemory_vs_Granulocyteseosinophils_p.val"                  
  [23] "BCellmemory_vs_Granulocytesneutrophils_p.val"                  
  [24] "BCellmemory_vs_Monocytesclassical_p.val"                       
  [25] "BCellmemory_vs_Monocytesintermediate_p.val"                    
  [26] "BCellmemory_vs_Monocytesnonclassical_p.val"                    
  [27] "BCellmemory_vs_NKCellsCD56bright_p.val"                        
  [28] "BCellmemory_vs_NKCellsCD56dim_p.val"                           
  [29] "BCellmemory_vs_Thrombocytes_p.val"                             
  [30] "BCellnaive_vs_BCellplasma_p.val"                               
  [31] "BCellnaive_vs_CD4TCellmTregs_p.val"                            
  [32] "BCellnaive_vs_CD4TCellnaive_p.val"                             
  [33] "BCellnaive_vs_CD4TCellnTregs_p.val"                            
  [34] "BCellnaive_vs_CD4TCellTcm_p.val"                               
  [35] "BCellnaive_vs_CD4TCellTem_p.val"                               
  [36] "BCellnaive_vs_CD4TCellTemra_p.val"                             
  [37] "BCellnaive_vs_CD4TCellTh1_p.val"                               
  [38] "BCellnaive_vs_CD4TCellTh17_p.val"                              
  [39] "BCellnaive_vs_CD4TCellTh2_p.val"                               
  [40] "BCellnaive_vs_CD8TCellnaive_p.val"                             
  [41] "BCellnaive_vs_CD8TCellTcm_p.val"                               
  [42] "BCellnaive_vs_CD8TCellTem_p.val"                               
  [43] "BCellnaive_vs_CD8TCellTemra_p.val"                             
  [44] "BCellnaive_vs_DendriticCD1c_p.val"                             
  [45] "BCellnaive_vs_DendriticCD304_p.val"                            
  [46] "BCellnaive_vs_Erythrocytes_p.val"                              
  [47] "BCellnaive_vs_GranulocytesBasophil_p.val"                      
  [48] "BCellnaive_vs_Granulocyteseosinophils_p.val"                   
  [49] "BCellnaive_vs_Granulocytesneutrophils_p.val"                   
  [50] "BCellnaive_vs_Monocytesclassical_p.val"                        
  [51] "BCellnaive_vs_Monocytesintermediate_p.val"                     
  [52] "BCellnaive_vs_Monocytesnonclassical_p.val"                     
  [53] "BCellnaive_vs_NKCellsCD56bright_p.val"                         
  [54] "BCellnaive_vs_NKCellsCD56dim_p.val"                            
  [55] "BCellnaive_vs_Thrombocytes_p.val"                              
  [56] "BCellplasma_vs_CD4TCellmTregs_p.val